<a href="https://colab.research.google.com/github/c444roline/dxc-ai-maintenance-assistant/blob/main/cleaning_dupontmolding_and_blowmolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing necessary libraries
import pandas as pd
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define the paths to the CSV files
dupont_blow_moulding_path = os.path.join('/content/drive/MyDrive/dxc_team/project_data/csv/', 'Dupont_blow_moulding_manual.csv')
blow_molding_basics_path = os.path.join('/content/drive/MyDrive/dxc_team/project_data/csv/', 'Blow_molding_basics.csv')

# Load the CLEANED_dupont_blow_moulding.csv into a pandas DataFrame with a different encoding
if os.path.exists(dupont_blow_moulding_path):
    df_1 = pd.read_csv(dupont_blow_moulding_path, encoding='ISO-8859-1')
    print("Dupont_blow_moulding_manual.csv loaded successfully.")
else:
    print("Dupont_blow_moulding_manual.csv not found.")

# Load the Blow_molding_basics.csv into a pandas DataFrame
if os.path.exists(blow_molding_basics_path):
    df_2 = pd.read_csv(blow_molding_basics_path, encoding='ISO-8859-1')
    print("Blow_molding_basics.csv loaded successfully.")
else:
    print("Blow_molding_basics.csv not found.")


Mounted at /content/drive
Dupont_blow_moulding_manual.csv loaded successfully.
Blow_molding_basics.csv loaded successfully.


In [ ]:
#cleaning Dupont_blow_moulding_manual.csv file! (in progress)

In [ ]:
import pandas as pd
import re

def extract_sections_with_indices(df):
    # Define sections and their respective subsections with starting rows
    sections_subsections = {
    "1 DuPont resins for blow moulding": [
            {"subsection": "1.1 Why blow mould engineering resins", "start_row": 6, "end_row": 131},
            {"subsection": "1.2 ZYTEL® nylon resins for blow moulding", "start_row": 139, "end_row": 515},
            {"subsection": "1.3 HYTREL® and CRASTIN® polyester resins for blow moulding", "start_row": 525, "end_row": 824}
    ],
    "2 Description of blow moulding processes": [
            {"subsection": "2.1 General", "start_row": 833, "end_row": 959},
            {"subsection": "2.2 Continuous extrusion machines", "start_row": 964, "end_row": 1101},
            {"subsection": "2.3 Accumulator head machines", "start_row": 1106, "end_row": 1270},
            {"subsection": "2.4 Co-extrusion and sequential 3-D blow moulding", "start_row": 1278, "end_row": 1982},
            {"subsection": "2.5 Injection blow moulding", "start_row": 1987, "end_row": 2277}
    ],
    "3 The blow moulding machine – Important considerations": [
        {"subsection": "3.1 Screw and barrel design", "start_row": 2283, "end_row": 2760},
        {"subsection": "3.2 Manifold/adapter design", "start_row": 2764, "end_row": 2912},
        {"subsection": "3.3 Accumulator and continuous extrusion head design", "start_row": 2920, "end_row": 3011},
        {"subsection": "3.4 Die/head tooling design", "start_row": 3016, "end_row": 3153},
        {"subsection": "3.5 Parison cutters", "start_row": 3157, "end_row": 3223},
        {"subsection": "3.6 Mould clamping force", "start_row": 3228, "end_row": 3294},
        {"subsection": "3.7 Temperature control", "start_row": 3298, "end_row": 3508},
        {"subsection": "3.8 Auxiliary equipment", "start_row": 3512, "end_row": 3733}
    ],
    "4 Machine operating conditions": [
        {"subsection": "4.1 Quick reference – Processing conditions", "start_row": 3750, "end_row": 3870},
        {"subsection": "4.2 Barrel temperatures", "start_row": 3874, "end_row": 4015},
        {"subsection": "4.3 Adapter, head and die temperature", "start_row": 4022, "end_row": 4387},
        {"subsection": "4.4 Accumulator push-out pressures and speeds", "start_row": 4394, "end_row": 4540},
        {"subsection": "4.5 Parison programming", "start_row": 4544, "end_row": 4639},
        {"subsection": "4.6 Mould temperature", "start_row": 4643, "end_row": 4747},
        {"subsection": "4.7 Start-up procedures", "start_row": 4751, "end_row": 5133},
        {"subsection": "4.8 Purging and shutdown", "start_row": 5138, "end_row": 5661},
        {"subsection": "4.9 Secondary operations: trimming, welding", "start_row": 5665, "end_row": 5937},
        {"subsection": "4.10 Special conditions for injection blow moulding and 'Pressblower' operation", "start_row": 5949, "end_row": 6113},
    ],
    "5 Handling of blow moulding resins": [
        {"subsection": "5.1 Effects of moisture", "start_row": 6124, "end_row": 6331},
        {"subsection": "5.2 Drying", "start_row": 6334, "end_row": 6539},
        {"subsection": "5.3 Regrind", "start_row": 6542, "end_row": 6832},
        {"subsection": "5.4 Bulk storage", "start_row": 6836, "end_row": 7107}
    ],
    "6 Mould design guidance": [
        {"subsection": "6.1 General", "start_row": 7114, "end_row": 7150},
        {"subsection": "6.2 Materials of construction", "start_row": 7155, "end_row": 7341},
        {"subsection": "6.3 Blow-up (draw) ratio", "start_row": 7346, "end_row": 7442},
        {"subsection": "6.4 Mould shrinkage allowances and part dimensions", "start_row": 7450, "end_row": 7640},
        {"subsection": "6.5 Pinch-off designs", "start_row": 7644, "end_row": 7786},
        {"subsection": "6.6 Other mould considerations", "start_row": 7791, "end_row": 8130}
    ]
    }

    # Prepare to store the extracted data
    data = []

    # Iterate over each section and its subsections
    for section, subsections in sections_subsections.items():
        for subsection_info in subsections:
            subsection = subsection_info["subsection"]
            start_row = subsection_info["start_row"]
            end_row = subsection_info["end_row"]

            # Extract text content for each subsection only from the first column
            text_block = df.iloc[start_row-1:end_row, 0].astype(str).tolist()  # Note that start_row-1 is used
            subsection_text = " ".join(text_block).strip()

            # Fix encoding issues by replacing problematic characters
            subsection_text = (
                subsection_text.encode('latin1').decode('utf-8')  # Fix common encoding issues
                .replace('â€“', '–')  # Fix dash issues
                .replace('Â°C', '°C')  # Fix degree symbols
                .replace('â€"', '“')  # Fix quotes
                .replace('â€', '”')  # Fix quotes
            )

            # Append the row with full section and subsection info and text
            data.append([section, subsection, subsection_text])

    # Create a structured DataFrame with section, subsection, and text columns
    organized_sections = pd.DataFrame(data, columns=['section', 'subsection', 'text'])

    return organized_sections

# Apply the function on the DataFrame
organized_sections = extract_sections_with_indices(df_1)

# Display the full organized data
organized_sections
# Save to CSV
output_path = '/content/drive/MyDrive/organized_Dupont_blow_moulding_manual.csv'
organized_sections.to_csv(output_path, index=False)
print(f"File saved to {output_path}")


File saved to /content/drive/MyDrive/organized_Dupont_blow_moulding_manual.csv


Not sure what this is below but i'm leaving it untouched LOL - jimena

In [ ]:
#cleaned Blow_molding_basics.csv file! (make sure each block of information is correct)
import pandas as pd

# Function to extract sections based on predefined row indices
def extract_sections_with_indices(df):
    # Define the start rows for each section, based on your provided indices
    sections = {
        "Introduction": 3,
        "Resins": 494,
        "Processes": 712,
        "Injection Blow Moulding": 932,
        "Injection Blow-Mold Tooling": 1605,
        "Troubleshooting Injection Blow Molding": 3297,
        "Setup for Injection Blow Tooling": 3424,
        "Extrusion Blow-Molding": 3743,
        "Formulas": 5211,
        "Die/Weight Swell": 5705,
        "Heat Extraction Load": 5814,
        "Metals Used in Blow Molds": 6826,
        "Shrinkage": 6929,
        "Venting": 7130,
        "Stretch Blow Molding": 8517,
        "Other Blow-Molding Operations": 10563,
        "BIBLIOGRAPHY": 11760,
        "GENERAL REFERENCES": 12187
    }

    # Sort sections by their start rows
    sorted_sections = sorted(sections.items(), key=lambda x: x[1])

    data = []
    for i in range(len(sorted_sections)):
        section_name, start_row = sorted_sections[i]

        # Determine the end row based on the next section's start or the end of the DataFrame
        if i + 1 < len(sorted_sections):
            end_row = sorted_sections[i + 1][1]
        else:
            end_row = len(df)

        # Extract text starting from the start_row (including it)
        section_text = " ".join(df.iloc[start_row-1:end_row, 0].astype(str).tolist()).strip()

        # Append the extracted section and its content
        data.append([section_name, section_text])

    return pd.DataFrame(data, columns=['section', 'text'])

# Apply the function
organized_sections = extract_sections_with_indices(df_2)

# Display and save the organized sections DataFrame
display(organized_sections)

# Save to CSV
output_path = '/content/drive/MyDrive/organized_blow_molding_basics.csv'
organized_sections.to_csv(output_path, index=False)
print(f"File saved to {output_path}")

NameError: name 'df_2' is not defined